# Evaluate heuristics with different values of k to compute k-shortest paths
Evaluate heuristics on different topologies for a range of k.
Can be evaluated for fixed length episodes or episodes that end on first blocking event.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
# Run the train script for each value of k
root = "/home/uceedoh"
data_directory = f"{root}/git/XLRON/data"
data_directory = f"{root}/git/XLRON/data/heuristic_benchmarks"
script_path = f"{root}/git/XLRON/xlron/train/train.py"
modulations_csv_filepath = f"{root}/git/XLRON/examples/modulations.csv"
def check_file(file_path):
    if not os.path.exists(file_path):
        return False
    try:
        with open(file_path, 'r') as file:
            return sum(1 for _ in file) > 2
    except IOError:
        return False

In [5]:
import jax
jax.devices()

In [3]:
env_type = "rmsa"
topologies = ["nsfnet_deeprmsa", "cost239_deeprmsa"]
heuristics = ['ksp_ff', 'ff_ksp', "ksp_bf", "bf_ksp", "kme_ff", "kmc_ff", "kmf_ff", "kca_ff"]
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
# Choice of traffic load - should be sufficient to cover wide range of blocking probs from <1% to 10%

commands = []
for topology in topologies:
    for heuristic in heuristics:
        for k in k_range:
            for load in load_range:
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}_k{k}_{load}.csv"
                if check_file(output_file):
                    print(f'Skipping file {output_file}')
                    pass
                else:
                    commands.append(f"python3 {script_path} --VISIBLE_DEVICES=3 --link_resources=100 --max_requests=1000 --max_timesteps=1000 --TOTAL_TIMESTEPS 10000 --NUM_ENVS 1 --NUM_SEEDS 100 --mean_service_holding_time=25 --ENV_WARMUP_STEPS 5000 --continuous_operation --noPLOTTING --ACTION_MASKING --EVAL_HEURISTIC --k {k} --env_type={env_type} --topology_name={topology} --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file} --load={load} --modulations_csv_filepath {modulations_csv_filepath}")

print(f"Total commands to run: {len(commands)}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Commands left: {len(commands) - i}")
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [ ]:
env_type = "rwa_lightpath_reuse"
num_seeds = 100
topologies = ["cost239", "nsfnet"]
heuristics = ['ksp_ff', 'ff_ksp', 'ksp_mu']#, 'ksp_mu_nonrel', 'ksp_mu_unique', 'mu_ksp', 'mu_ksp_nonrel', 'mu_ksp_unique']
num_requests_nsfnet = [1e4, 1e4, 1.5e4, 2e4, 2.5e4]
num_requests_cost239 = [2e4, 2e4, 2.5e4, 3e4, 3.5e4]
k_range = range(1, 11)

commands = []
for topology in topologies:
    num_requests_list = num_requests_nsfnet if topology=="nsfnet" else num_requests_cost239
    for heuristic in heuristics:
        for i, num_requests in enumerate(num_requests_list):
            first_blocking = (i == 0)
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}_k{k}_{num_requests:.0f}{'_firstblocking' if first_blocking else ''}.csv"
                if check_file(output_file):
                    print(f'Skipping file {output_file}')
                    pass
                commands.append(f"python3 ../xlron/train/train.py --VISIBLE_DEVICES=3 --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests={num_requests} --max_timesteps={num_requests} --values_bw=100 --TOTAL_TIMESTEPS {int(num_requests)} --NUM_ENVS 1 --NUM_SEEDS {num_seeds} --ACTION_MASKING --incremental_loading {'--end_first_blocking' if first_blocking else ''} --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file}")

print(f"Total commands to run: {len(commands)}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Commands left: {len(commands) - i}")
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [ ]:
# Read each of the generated files, get the mean accepted services (or metric of choice) and plot

for end_first_blocking in [False, True]:
    for topology in ["cost239", "nsfnet"]:
        accepted_services = []
        labels = []
        for heuristic in [
            'ksp_ff', 
            'ff_ksp', 
            #'ksp_mu', 
            #'mu_ksp', 
            "ksp_mu_alt", 
            "mu_ksp_alt"]:
            mean_accepted_services = []
            std_accepted_services = []
            for k in range (1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_firstblocking' if end_first_blocking else ''}.csv"
                df = pd.read_csv(output_file)
                accepted_services.append(df['accepted_services'])
                mean_accepted_services.append(df['accepted_services'].mean())
                std_accepted_services.append(df['accepted_services'].std())
                print(f"Mean accepted services for {heuristic}{k}: {df['accepted_services'].mean()}")
                labels.append(f"{heuristic}{k}")
            plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
            plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel("Mean accepted services")
        plt.legend()
        plt.grid()
        plt.title(f"{topology}-{'first blocking' if end_first_blocking else 'fixed length'}")
        plt.show()
        # Plot as a boxplot
        # plt.boxplot(accepted_services, labels=labels)
        # plt.xlabel("k")
        # plt.ylabel("Mean accepted services")
        # plt.legend()
        # plt.show()

In [ ]:
# Same but for first blocking
commands = []
for topology in ["cost239", "nsfnet"]:
    for num_requests in [1e4, 2e4, 3e4, 4e4]:
        for heuristic in ['ksp_ff', 'ff_ksp', 'ksp_mu', 'mu_ksp', "ksp_mu_alt", "mu_ksp_alt"]:
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{str(int(num_requests))}.csv"
                commands.append(f"python3 ../xlron/train/train.py --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests={num_requests} --max_timesteps={num_requests} --values_bw=100 --TOTAL_TIMESTEPS 120000 --NUM_ENVS 1 --NUM_SEEDS 1 --ACTION_MASKING --incremental_loading  --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file}")
            
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [ ]:

for topology in ["cost239", "nsfnet"]:
    for num_requests in [1e4, 2e4, 3e4, 4e4]:
        accepted_services = []
        labels = []
        for heuristic in [
            'ksp_ff', 
            'ff_ksp', 
            'ksp_mu', 
            'mu_ksp', 
            "ksp_mu_alt", 
            "mu_ksp_alt"]:
            mean_accepted_services = []
            std_accepted_services = []
            for k in range (1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{str(int(num_requests))}.csv"
                df = pd.read_csv(output_file)
                accepted_services.append(df['accepted_services'])
                mean_accepted_services.append(df['accepted_services'].mean())
                std_accepted_services.append(df['accepted_services'].std())
                print(f"Mean accepted services for {heuristic}{k}: {df['accepted_services'].mean()}")
                labels.append(f"{heuristic}{k}")
            plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
            plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel("Mean accepted services")
        plt.legend()
        plt.grid()
        plt.title(f"{topology}-{num_requests}")
        plt.show()

In [ ]:
# Same but for dynamic RMSA environment
env_type = "rmsa"
commands = []
for topology in ["nsfnet_deeprmsa"]:
    for weight in [True, False]:
        for heuristic in ['ksp_ff', 'ff_ksp', 'kmf_ff', 'kmc_ff']:#, 'ksp_mu', 'mu_ksp']:
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_weighted' if weight else ''}_deeprmsa.csv"
                commands.append(f"python3 ../xlron/train/train.py --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests=1e3 --max_timesteps=1e3 --load=250 --mean_service_holding_time=25 --TOTAL_TIMESTEPS 120000 --NUM_ENVS 1 --NUM_SEEDS 1 --ACTION_MASKING  --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file} --truncate_holding_time --ENV_WARMUP_STEPS 3000 --continuous_operation {'--weight weight' if weight else ''}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [ ]:
env_type = "rmsa"
commands = []
for metric in ["service_blocking_probability", "bitrate_blocking_probability"]:
    for topology in ["nsfnet_deeprmsa"]:
        for weight in [True, False]:
            accepted_services = []
            labels = []
            for heuristic in ['ksp_ff', 'ff_ksp', 'kmc_ff']:#, "kmf_ff"]:#, 'mu_ksp', 'ksp_mu']: 
                mean_accepted_services = []
                std_accepted_services = []
                for k in range (1, 11):
                    output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_weighted' if weight else ''}_deeprmsa.csv"
                    df = pd.read_csv(output_file)
                    accepted_services.append(df[metric])
                    mean_accepted_services.append(df[metric].mean())
                    std_accepted_services.append(df[metric].std())
                    print(f"Mean {metric} for {heuristic}{k}: {df[metric].mean()}")
                    labels.append(f"{heuristic}{k}")
                plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
                plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel(f"Mean {metric}")
        plt.legend()
        plt.grid()
        plt.show()

In [10]:
# To find the effect of truncating the holding time ot be less than 2*mean
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set up Helvetica font
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans', 'Bitstream Vera Sans', 'sans-serif']
plt.rcParams.update({'font.size': 20})

def deeprmsa_sample(mean):
    holding_time = np.random.exponential(mean)
    while holding_time > 2*mean:
        holding_time = np.random.exponential(mean)
    return holding_time

# plot the distribution of holding times
mean = 25
n_samples = 1000000
deeprmsa = np.array([deeprmsa_sample(mean) for i in range(n_samples)])
rmsa = np.array([np.random.exponential(mean) for i in range(n_samples)])
max_ht = mean#max(max(deeprmsa), max(rmsa))
deeprmsa_norm = deeprmsa/max_ht
rmsa_norm = rmsa/max_ht
mean_ht_truncated = np.mean(deeprmsa_norm)
mean_ht = np.mean(rmsa_norm)
bins = np.arange(0, 10.01, 0.01)
weights = np.ones_like(rmsa_norm) / len(rmsa_norm)
plt.hist(deeprmsa_norm, bins=bins, label="Truncated", alpha=0.5, density=True)
plt.hist(rmsa_norm, bins=bins, label="Not truncated", alpha=0.5, density=True)
plt.axvline(mean_ht_truncated, color='b', linestyle='dashed', linewidth=1, label="Mean truncated")
plt.axvline(mean_ht, color='r', linestyle='dashed', linewidth=1, label="Mean not truncated")
# Legend with 12 text size
plt.legend(fontsize=11)
plt.xlabel("Normalised holding time", size=14)
plt.ylabel("Density", size=14)
plt.xlim([0,6])
# Make x and y ticks larger
plt.xticks(size=12)
plt.yticks(size=12)
# Add a box saying the ratio of the means
# Draw an arrow pointing from mean_ht to mean_ht_truncated
plt.arrow(mean_ht, 0.8, -mean_ht+mean_ht_truncated, 0, head_width=0.05, head_length=0.1, fc='k', ec='k', length_includes_head=True)
plt.text(mean_ht+0.15, 0.75, f"{int(((mean_ht-mean_ht_truncated) / mean_ht)*100)}% decrease in\nmean holding time", bbox=dict(facecolor='white', alpha=0.5), size=11)
plt.tight_layout()
plt.savefig("truncation.png")
plt.show()


In [7]:
import numpy as np
import matplotlib.pyplot as plt
ratios = []
for mean_holding_time in [5,10,15,20,25,30]:
    hts = []
    for i in range(10000000):
        holding_time = deeprmsa_sample(mean_holding_time)
        hts.append(holding_time)
    real_mean_ht = np.mean(hts)
    ratio = real_mean_ht/mean_holding_time
    ratios.append(ratio)
    print(f"Real mean: {real_mean_ht}")
    print(f"Ration of real to expected: {real_mean_ht/mean_holding_time}")
real_mean = np.mean(ratios)
real_std = np.sqrt(np.sum(((np.array(ratios)-real_mean)**2)/(len(ratios)-1)))
print(real_mean)
print(real_std)

In [11]:
# Investigate steady state condition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
mean_service_holding_time = 25
arrival_rate = 10
load = mean_service_holding_time*arrival_rate
num_requests = 100000
active_services = []
current_time = 0

results = []
for repeats in range(1000):
    for load in np.arange(50,1050,50):
        for arrival_rate in [5, 10, 15, 20, 25]:
            mean_service_holding_time = load/arrival_rate
        #for mean_service_holding_time in [5, 10, 15, 20, 25, 30]:
        #    load = mean_service_holding_time*arrival_rate
            active_services = []
            current_time = 0
            for i in range(num_requests):
                current_time += np.random.exponential(1/arrival_rate)
                holding_time = np.random.exponential(mean_service_holding_time)
                active_services.append(current_time + holding_time)
                active_services = [x for x in active_services if x > current_time]
                if len(active_services) >= load:
                    data = {"arrival_rate": arrival_rate, "mean_service_holding_time": mean_service_holding_time, "load": load, "current_time": current_time, "active_services": len(active_services), "num_requests": i}
                    results.append(data)
                    break
df = pd.DataFrame(results)
# df_mean = df.groupby(["arrival_rate", "mean_service_holding_time"]).mean()
# df_mean = df_mean.reset_index()
# df_std = df.groupby(["arrival_rate", "mean_service_holding_time"]).std()
# df_std = df_std.reset_index()
# df = df_mean.merge(df_std, on=["arrival_rate", "mean_service_holding_time"], suffixes=('_mean', '_std'))
df

In [12]:
# Plot the mean steady state requests against load and fit a line
df_mean = df.groupby("load").mean()
df_mean = df_mean.reset_index()
df_std = df.groupby("load").std()
df_std = df_std.reset_index()
df_mean = df_mean.merge(df_std, on="load", suffixes=('_mean', '_std'))


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Get series of num_requests for each load
requests = [df[df["load"] == load]["num_requests"] for load in sorted(df["load"].unique())]
loads = sorted(df["load"].unique())

# Create box plot
bp = ax.boxplot(requests, positions=loads, widths=20, patch_artist=True, showfliers=False, showmeans=True)

# Customize box appearance
for box in bp['boxes']:
    box.set(facecolor='white', edgecolor='black')
for whisker in bp['whiskers']:
    whisker.set(color='black')
for cap in bp['caps']:
    cap.set(color='black')
for median in bp['medians']:
    median.set(color='red')

# Extract upper whisker values
upper_whiskers = [item.get_ydata()[1] for item in bp['whiskers'][1::2]]

# Fit a line to the upper whiskers
coeffs = np.polyfit(loads, upper_whiskers, 1)
print(coeffs)
poly = np.poly1d(coeffs)

# Calculate R-squared
y_pred = poly(loads)
y_mean = np.mean(upper_whiskers)
ss_tot = np.sum((upper_whiskers - y_mean)**2)
ss_res = np.sum((upper_whiskers - y_pred)**2)
r_squared = 1 - (ss_res / ss_tot)

# Plot the fitted line with R-squared in the label
ax.plot(loads, poly(loads), color='r', linestyle='--', 
        label=f'Fitted line: y = {coeffs[0]:.2f}x {"+" if coeffs[1]>0 else "-"} {abs(coeffs[1]):.0f}\nR² = {r_squared:.4f}')

ax.set_xlabel("Traffic load", size=18)
ax.set_ylabel("Requests until steady state", size=18)
ax.legend(fontsize=16)
plt.xticks(size=16)
plt.yticks(size=16)

# Set x-axis ticks to show load/100
ax.set_xticks(loads[1::2])  # Show every other tick to avoid crowding
ax.set_xticklabels([f'{load:.0f}' for load in loads[1::2]])

plt.tight_layout()
plt.savefig("steady_state_boxplots.png")
plt.show()

In [11]:
# Investigate the effect of truncating the holding time on traffic load
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def deeprmsa_sample(mean):
    holding_time = np.random.exponential(mean)
    while holding_time > 2*mean:
        holding_time = np.random.exponential(mean)
    return holding_time

num_requests = 10000
results = []

for repeats in range(10):
    for load in np.arange(50,1050,50):
        for arrival_rate in [5, 10, 15, 20, 25]:
            mean_service_holding_time = load/arrival_rate
            active_services = []
            current_time = 0
            for i in range(load*7):
                current_time += np.random.exponential(1/arrival_rate)
                holding_time = deeprmsa_sample(mean_service_holding_time)
                active_services.append(current_time + holding_time)
                active_services = [x for x in active_services if x > current_time]
            for i in range(num_requests):
                current_time += np.random.exponential(1/arrival_rate)
                holding_time = deeprmsa_sample(mean_service_holding_time)
                active_services.append(current_time + holding_time)
                active_services = [x for x in active_services if x > current_time]
                data = {"arrival_rate": arrival_rate, "mean_service_holding_time": mean_service_holding_time, "load": load, "current_time": current_time, "active_services": len(active_services), "num_requests": i}
                results.append(data)
df = pd.DataFrame(results)

In [13]:
# For each load and arrivla rate, plot active services vs i
# Plot the mean steady state requests against load and fit a line
df_mean = df.groupby(["load", "arrival_rate"]).mean()
df_mean = df_mean.reset_index()
df_std = df.groupby(["load", "arrival_rate"]).std()
df_std = df_std.reset_index()
df_grouped = df_mean.merge(df_std, on=["load", "arrival_rate"], suffixes=('_mean', '_std'))
df_grouped = df_grouped.reset_index()
df_grouped

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib as mpl

# Set up Helvetica font
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans', 'Bitstream Vera Sans', 'sans-serif']
plt.rcParams.update({'font.size': 20})

# Assuming df_grouped is already created as in your code

fig, ax = plt.subplots(figsize=(10, 6))

# Create scatter plot with error bars
for arrival_rate in df_grouped['arrival_rate'].unique():
    data = df_grouped[df_grouped['arrival_rate'] == arrival_rate]
    ax.errorbar(data['load'], data['active_services_mean'], 
                yerr=data['active_services_std'], 
                fmt='o', capsize=5, label=f'Arrival Rate: {arrival_rate}')

# Fit a line to all data points
x = df_grouped['load']
y = df_grouped['active_services_mean']
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Plot the fitted line
line = slope * x + intercept
ax.plot(x, line, color='r', linestyle='--', 
        label=f'Fitted line: y = {slope:.2f}x {"+" if intercept>0 else "-"} {abs(intercept):.0f}\nR² = {r_value**2:.4f}')

ax.set_xlabel("Expected traffic load [Erlang]", size=18)
ax.set_ylabel("Active Services", size=18)
ax.legend(fontsize=16)
plt.xticks(size=16)
plt.yticks(size=16)

# Set x-axis ticks to show load/100
ax.set_xticks(np.arange(100,1100,100))  # Show every other tick to avoid crowding

# Add a box underneath the line saying actual traffic 69% of expected
# Draw an arrow pointing from the line to the box
plt.arrow(680, 310, 0, 600*0.25, head_width=10, head_length=10, fc='k', ec='k', length_includes_head=True)
plt.text(580, 280, "Actual traffic 69% of expected", bbox=dict(facecolor='white', alpha=0.5), size=16)


plt.tight_layout()
plt.savefig("accepted_services_vs_load.png")
plt.show()

# Print additional statistics
print(f"Slope: {slope:.4f}")
print(f"Intercept: {intercept:.4f}")
print(f"R-squared: {r_value**2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Standard Error: {std_err:.4f}")